In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
sc.stop()

In [0]:
from pyspark import SparkContext, SparkConf

In [0]:
from pyspark.sql import SQLContext
conf = SparkConf().setAppName("Spark_Proj").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf)
sqlcontext = SQLContext(sc)

/databricks/spark/python/pyspark/sql/context.py:82: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [0]:
transaction_df = spark.read.csv("dbfs:/FileStore/tables/all_transaction_1.csv", header = True, inferSchema =True)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import date_format,year,col,to_timestamp


In [0]:
transaction_df.printSchema()
transaction_df.show(10, truncate = False)

root
 |-- ProductCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- SubCategory: string (nullable = true)
 |-- ActualPrice: double (nullable = true)
 |-- Marginpercentage: integer (nullable = true)
 |-- Discountpercentage: integer (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- TransactionAmount: double (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- Mode: string (nullable = true)
 |-- InvoiceNumber: integer (nullable = true)
 |-- CustomerId: string (nullable = true)
 |-- Country: string (nullable = true)

+------------------------------------+---------------------------------------------------------------------------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+
|ProductCode                  

In [0]:
transaction_df1=transaction_df.withColumn("Year",year(transaction_df.InvoiceDate)).withColumn("Month",month(transaction_df.InvoiceDate))

In [0]:
transaction_df1.createTempView("amazon_transaction_data")

In [0]:
transaction_df1.select("Brand","SubCategory","Month","Year").show()

+------------+-------------------+-----+----+
|       Brand|        SubCategory|Month|Year|
+------------+-------------------+-----+----+
|      landau|  all women's shoes|    1|2015|
|       adult|  all women's shoes|    1|2015|
|    woolrich|           athletic|    1|2015|
|       style|      women's shoes|    1|2015|
|         two|  all women's shoes|    1|2015|
|     michael|          necklaces|    1|2015|
|        bali|    shoes & jewelry|    1|2015|
|dr. scholl's|  all women's shoes|    1|2015|
| coloriffics|women's accessories|    1|2015|
|  timberland|       shoes & bags|    1|2015|
|      bamboo|  all women's shoes|    1|2015|
|   evolatree|women's accessories|    1|2015|
|         bar|women's accessories|    1|2015|
|     michael|  all women's shoes|    1|2015|
|      excell|   women's clothing|    1|2015|
|        vans|      women's jeans|    1|2015|
|         nwt|      women's shoes|    1|2015|
|        toms|  all women's shoes|    1|2015|
|       earth|              boots|

1 a)Find the top five performing mobile brands for the months of November, December and January r each year.

In [0]:
spark.sql("with sub as(Select Brand, sum(Quantity) as TotalProductSold, Month,Year, ROW_NUMBER() over(Partition by Year, Month order by sum(Quantity) DESC) as Rank from amazon_transaction_data where SubCategory = 'mobile' and Month in (11,12,1) group by Year, Month, Brand)select Brand, Month, Year, TotalProductSold from sub where rank < 6").show(50)

+--------+-----+----+----------------+
|   Brand|Month|Year|TotalProductSold|
+--------+-----+----+----------------+
| samsung|    1|2015|             377|
|   apple|    1|2015|             223|
|   nokia|    1|2015|             168|
|     blu|    1|2015|             148|
|motorola|    1|2015|             127|
| samsung|   11|2015|             448|
|     blu|   11|2015|             234|
|   apple|   11|2015|             200|
|   nokia|   11|2015|             185|
|      lg|   11|2015|             165|
| samsung|   12|2015|             517|
|     blu|   12|2015|             249|
|   nokia|   12|2015|             212|
|   apple|   12|2015|             184|
|     htc|   12|2015|             158|
| samsung|    1|2016|             361|
|   apple|    1|2016|             216|
|     blu|    1|2016|             171|
|   nokia|    1|2016|             159|
|      lg|    1|2016|             145|
| samsung|   11|2016|             410|
|     blu|   11|2016|             204|
|   apple|   11|2016|    

1b)Find the top five selling products for the months of November, December and January for each year.

In [0]:
spark.sql("with sub as(Select subcategory, sum(Quantity) as ProductSold, Month,Year, ROW_NUMBER() over(Partition by Year, Month order by sum(Quantity) DESC) as Rank from amazon_transaction_data where Month in (11,12,1) group by Year, Month, subcategory)select subcategory, Month, Year, ProductSold from sub where rank < 6").show(truncate= False)

+-----------------+-----+----+-----------+
|subcategory      |Month|Year|ProductSold|
+-----------------+-----+----+-----------+
|all women's shoes|1    |2015|2891       |
|mobile           |1    |2015|2255       |
|men's shoes      |1    |2015|1835       |
|shoes & jewelry  |1    |2015|955        |
|women's shoes    |1    |2015|829        |
|mobile           |11   |2015|2701       |
|men's shoes      |11   |2015|2186       |
|all women's shoes|11   |2015|2044       |
|shoes & jewelry  |11   |2015|870        |
|women's shoes    |11   |2015|586        |
|all women's shoes|12   |2015|2970       |
|mobile           |12   |2015|2935       |
|men's shoes      |12   |2015|2010       |
|shoes & jewelry  |12   |2015|919        |
|women's shoes    |12   |2015|911        |
|mobile           |1    |2016|2262       |
|all women's shoes|1    |2016|2243       |
|men's shoes      |1    |2016|2092       |
|shoes & jewelry  |1    |2016|946        |
|women's shoes    |1    |2016|622        |
+----------

2.Give valuable insights to the investors

In [0]:
amazon_df1=transaction_df1.withColumn("Quarter",quarter(transaction_df1.InvoiceDate))
amazon_df1.createOrReplaceTempView("Data_Table1")

2a. Number of products sold along with revenue aggregated quarterly

In [0]:
spark.sql("select Year,Quarter,category,sum(quantity) as totalProductsSold ,sum(TransactionAmount) as QuarterRevenueAmount from Data_Table1 group by Year,Quarter,category order by Year,Quarter,totalProductsSold Desc").show()

+----+-------+-------------+-----------------+--------------------+
|Year|Quarter|     category|totalProductsSold|QuarterRevenueAmount|
+----+-------+-------------+-----------------+--------------------+
|2015|      1|     clothing|            32683|        1.89308525E8|
|2015|      1|  electronics|             7872|        1.10534375E8|
|2015|      1|        shoes|             2134|           8619415.0|
|2015|      1|apple carplay|                4|             86240.0|
|2015|      2|     clothing|            32481|        1.87920992E8|
|2015|      2|  electronics|             7434| 2.23020102342676E14|
|2015|      2|        shoes|             2154|         1.2181617E7|
|2015|      3|     clothing|            33517|        2.08341854E8|
|2015|      3|  electronics|             7497|         9.8171612E7|
|2015|      3|        shoes|             2210|          1.106511E7|
|2015|      4|     clothing|            33537|        1.80563516E8|
|2015|      4|  electronics|             9917|  

2b. Number of products sold in each category aggregated quarterly

In [0]:
spark.sql("select Year,Quarter,subcategory,sum(quantity) as totalProductsSold from Data_Table1 group by Year,Quarter,subcategory order by Year,Quarter,totalProductsSold Desc").show(truncate =False)

+----+-------+----------------------------------+-----------------+
|Year|Quarter|subcategory                       |totalProductsSold|
+----+-------+----------------------------------+-----------------+
|2015|1      |all women's shoes                 |7155             |
|2015|1      |mobile                            |6487             |
|2015|1      |men's shoes                       |5854             |
|2015|1      |shoes & jewelry                   |2587             |
|2015|1      |women's shoes                     |2134             |
|2015|1      |all men's shoes                   |1338             |
|2015|1      |athletic                          |623              |
|2015|1      |women's accessories               |619              |
|2015|1      |sunglasses & eyewear accessories  |592              |
|2015|1      |jewelry                           |530              |
|2015|1      |office safety & security          |492              |
|2015|1      |all women's clothing              

2c A number of products sold for each brand aggregated quarterly.

In [0]:
spark.sql("select Brand,year,quarter,sum(Quantity) as Num_Product_Sold from Data_Table1 group by year,Brand,quarter order by Year,Quarter,Num_Product_Sold desc").show()

+----------+----+-------+----------------+
|     Brand|year|quarter|Num_Product_Sold|
+----------+----+-------+----------------+
|      nike|2015|      1|            2642|
|   samsung|2015|      1|            1174|
|     apple|2015|      1|             646|
|       new|2015|      1|             634|
|      puma|2015|      1|             618|
|    reebok|2015|      1|             539|
|       blu|2015|      1|             533|
|      vans|2015|      1|             457|
|     nokia|2015|      1|             441|
|     yu&yu|2015|      1|             384|
|      sony|2015|      1|             382|
|  skechers|2015|      1|             380|
|    adidas|2015|      1|             348|
|       htc|2015|      1|             348|
|        lg|2015|      1|             325|
|      toms|2015|      1|             312|
|  motorola|2015|      1|             300|
|  converse|2015|      1|             268|
|blackberry|2015|      1|             260|
|pleaserusa|2015|      1|             242|
+----------

3.Find products that generated profit
 
 3a Top 5 profit margin generating products for a given month.

In [0]:
spark.sql("with sub as(select subcategory,Month,year,sum(if((Marginpercentage-Discountpercentage)>0,1,0)) as profit ,ROW_NUMBER() over(Partition by Year, Month order by sum(if((Marginpercentage-Discountpercentage)>0,1,0)) DESC) as Rank from amazon_transaction_data group by Year, Month, subcategory)select subcategory, Month, Year, profit from sub where rank < 6 order by year,Month,profit desc").show(truncate= False)

#using Category
spark.sql("with sub as(select category,Month,year,sum(if((Marginpercentage-Discountpercentage)>0,1,0)) as profit ,ROW_NUMBER() over(Partition by Year, Month order by sum(if((Marginpercentage-Discountpercentage)>0,1,0)) DESC) as Rank from amazon_transaction_data group by Year, Month, category)select category, Month, Year, profit from sub where rank < 6 order by year,Month,profit desc").show()


+-----------------+-----+----+------+
|subcategory      |Month|Year|profit|
+-----------------+-----+----+------+
|all women's shoes|1    |2015|328   |
|mobile           |1    |2015|260   |
|men's shoes      |1    |2015|225   |
|shoes & jewelry  |1    |2015|136   |
|women's shoes    |1    |2015|116   |
|mobile           |2    |2015|239   |
|all women's shoes|2    |2015|215   |
|men's shoes      |2    |2015|178   |
|women's shoes    |2    |2015|82    |
|shoes & jewelry  |2    |2015|76    |
|men's shoes      |3    |2015|254   |
|all women's shoes|3    |2015|251   |
|mobile           |3    |2015|231   |
|shoes & jewelry  |3    |2015|110   |
|women's shoes    |3    |2015|77    |
|all women's shoes|4    |2015|279   |
|men's shoes      |4    |2015|220   |
|mobile           |4    |2015|171   |
|shoes & jewelry  |4    |2015|106   |
|women's shoes    |4    |2015|87    |
+-----------------+-----+----+------+
only showing top 20 rows

+-------------+-----+----+------+
|     category|Month|Year|pr

4 Brands that need 

4a Brand wise segregation of best performing categories profits wise

In [0]:
spark.sql("select Brand,SubCategory,sum(if((Marginpercentage-Discountpercentage)>0,1,0)) as profit from amazon_transaction_data group by Brand,SubCategory order by profit desc").show()

#profit = sp - cp ; cp = (sp*100)/(100+margin%)
#spark.sql("select brand,SubCategory, (ActualPrice- CostPrice) as profit from (select brand,SubCategory,((ActualPrice * 100)/(100+Marginpercentage)) as CostPrice from amazon_transaction_data group by brand,SubCategory) group by brand,SubCategory order by profit desc").show(truncate=False)

+----------+-----------------+------+
|     Brand|      SubCategory|profit|
+----------+-----------------+------+
|   samsung|           mobile|  1591|
|      nike|      men's shoes|  1296|
|     apple|           mobile|   794|
|       blu|           mobile|   750|
|        lg|           mobile|   598|
|     nokia|           mobile|   561|
|blackberry|           mobile|   537|
|       htc|           mobile|   430|
|      sony|           mobile|   426|
|  motorola|           mobile|   371|
|      nike|  shoes & jewelry|   299|
|      puma|      men's shoes|   275|
|    huawei|           mobile|   254|
|     yu&yu|all women's shoes|   234|
|      vans|      men's shoes|   223|
|      nike|all women's shoes|   216|
|    reebok|      men's shoes|   214|
|      nike|  all men's shoes|   213|
|       new|      men's shoes|   206|
|    adidas|      men's shoes|   190|
+----------+-----------------+------+
only showing top 20 rows



4 b Brand wise segregation of least performing categories orders wise

In [0]:
spark.sql("select Brand,subcategory,sum(Quantity) as products_sold from amazon_transaction_data group by Brand,subcategory order by products_sold asc").show()

+---------+----------------------------+-------------+
|Brand    |subcategory                 |products_sold|
+---------+----------------------------+-------------+
|hailey   |women's shoes               |1            |
|liz      |shoes & handbags            |1            |
|cooler   |auto & tires                |1            |
|converse |other wholesale women's accs|1            |
|platinum |jewelry                     |1            |
|luxury   |women's clothing            |1            |
|nwt      |men's work shoes & boots    |1            |
|basic    |shoes & jewelry             |1            |
|capezio  |waist packs                 |1            |
|silvertoe|shoes & jewelry             |1            |
|patrick  |men's clothing              |1            |
|totes    |women's dress shoes         |1            |
|tommy    |intimates & sleepwear       |1            |
|parisa   |all women's shoes           |1            |
|luckers  |all women's shoes           |1            |
|deer     

5Find the products that should be part of the lighting deals.

a Find the top five selling products in a given month

In [0]:
spark.sql("with sub as(Select subcategory, sum(Quantity) as ProductSold, Month,Year, ROW_NUMBER() over(Partition by Year, Month order by sum(Quantity) DESC) as Rank from amazon_transaction_data group by Year, Month, subcategory)select subcategory, Month, Year, ProductSold from sub where rank < 6").show()

+-----------------+-----+----+-----------+
|      subcategory|Month|Year|ProductSold|
+-----------------+-----+----+-----------+
|all women's shoes|    1|2015|       2891|
|           mobile|    1|2015|       2255|
|      men's shoes|    1|2015|       1835|
|  shoes & jewelry|    1|2015|        955|
|    women's shoes|    1|2015|        829|
|           mobile|    2|2015|       2131|
|all women's shoes|    2|2015|       1993|
|      men's shoes|    2|2015|       1729|
|  shoes & jewelry|    2|2015|        646|
|    women's shoes|    2|2015|        592|
|      men's shoes|    3|2015|       2290|
|all women's shoes|    3|2015|       2271|
|           mobile|    3|2015|       2101|
|  shoes & jewelry|    3|2015|        986|
|    women's shoes|    3|2015|        713|
|all women's shoes|    4|2015|       2236|
|      men's shoes|    4|2015|       2088|
|           mobile|    4|2015|       1680|
|  shoes & jewelry|    4|2015|        892|
|    women's shoes|    4|2015|        677|
+----------

5 b Find the top five products having the highest margin in a given month

In [0]:
spark.sql("with sub as(select subcategory,Month,year,sum(Marginpercentage) as highest_margin ,ROW_NUMBER() over(Partition by Year, Month order by sum(Marginpercentage) DESC) as Rank from amazon_transaction_data group by Year, Month, subcategory)select subcategory, Month, Year,highest_margin from sub where rank < 6 order by year,Month,highest_margin desc").show(truncate= False)


+-----------------+-----+----+--------------+
|subcategory      |Month|Year|highest_margin|
+-----------------+-----+----+--------------+
|all women's shoes|1    |2015|16910         |
|mobile           |1    |2015|13210         |
|men's shoes      |1    |2015|10940         |
|shoes & jewelry  |1    |2015|5690          |
|women's shoes    |1    |2015|5400          |
|mobile           |2    |2015|11980         |
|all women's shoes|2    |2015|11160         |
|men's shoes      |2    |2015|9200          |
|shoes & jewelry  |2    |2015|3690          |
|women's shoes    |2    |2015|3530          |
|men's shoes      |3    |2015|13270         |
|all women's shoes|3    |2015|13140         |
|mobile           |3    |2015|12260         |
|shoes & jewelry  |3    |2015|5440          |
|women's shoes    |3    |2015|3950          |
|all women's shoes|4    |2015|14060         |
|men's shoes      |4    |2015|11930         |
|mobile           |4    |2015|8650          |
|shoes & jewelry  |4    |2015|5270

5c Find the five lowest selling products in a given month.

In [0]:
spark.sql("with sub as(select subcategory,Month,year,sum(quantity) as low_selling ,ROW_NUMBER() over(Partition by Year, Month order by sum(quantity)) as Rank from amazon_transaction_data group by subcategory,Year, Month)select subcategory, Month, Year,low_selling from sub where rank < 6 order by year,Month,low_selling").show(truncate= False)


+----------------------------------------------------------+-----+----+-----------+
|subcategory                                               |Month|Year|low_selling|
+----------------------------------------------------------+-----+----+-----------+
|women's running shoes                                     |1    |2015|1          |
|link                                                      |1    |2015|1          |
|wristwatch bands                                          |1    |2015|1          |
|scuff                                                     |1    |2015|1          |
|masks & peels                                             |1    |2015|1          |
|see more saxx 9699 mens vibe orange striped modern fit ...|2    |2015|1          |
|neutral                                                   |2    |2015|1          |
|occupational                                              |2    |2015|1          |
|coveralls & suits                                         |2    |2015|1    